# Anti Spoofing CNN

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tqdm import tqdm

## Data Download

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("faber24/lcc-fasd")

print("Path to dataset files:", path)

/home/unai/Documents/Uni/Año4/Deusto/SI/SIEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1


In [3]:
import os

for entry in os.scandir(path):
    if entry.is_dir:
        for entry2 in os.scandir(entry.path):
            print(entry2.path)
    

/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_evaluation
/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_training
/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_development


In [4]:
data_splits_paths = {
    "training":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_training",
    "evaluation":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_evaluation",
    "development":"/home/unai/.cache/kagglehub/datasets/faber24/lcc-fasd/versions/1/LCC_FASD/LCC_FASD_development"
}


## Creating Dataloader

In [5]:
data = datasets.ImageFolder(data_splits_paths["training"])
data.class_to_idx

{'real': 0, 'spoof': 1}

In [12]:
transform = transforms.Compose([
    transforms.Resize(256),  # Resize images to 224x224
    transforms.CenterCrop(224),
    transforms.ToTensor(),          # Convert images to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Assuming you have a directory with 'train' and 'test' folders and each folder has class subfolders
train_dataset = datasets.ImageFolder(root=data_splits_paths['training'], transform=transform)
test_dataset = datasets.ImageFolder(root=data_splits_paths['evaluation'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, shuffle=False)

## Training

In [7]:
def simple_train_model(model,criterion,optimizer,epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):  # loop over the dataset multiple times
        model.train(True)
        running_loss = 0.0
        running_f1 = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)

            optimizer.zero_grad()  # zero the parameter gradients
            
            outputs = model(inputs)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            running_loss += loss
            optimizer.step()
        
            y_pred, y_true = np.round(outputs.detach().cpu().numpy(), decimals=0), labels.detach().cpu().numpy()
            running_f1 += f1_score(y_pred=y_pred, y_true=y_true)
            running_acc += accuracy_score(y_pred=y_pred, y_true=y_true)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/i:.4f}, F1 Score: {running_f1/i:.4f}, Accuracy: {running_acc/i:.4f}')



In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [10]:
class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [8]:
class NotsoDeepCNN(nn.Module):
    def __init__(self):
        super(NotsoDeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 300)
        self.fc2 = nn.Linear(300, 40)
        self.fc3 = nn.Linear(40, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [25]:
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 14.7456, F1 Score: 0.9228, Accuracy: 0.8561
Epoch 2/5, Loss: 14.8023, F1 Score: 0.9228, Accuracy: 0.8558
Epoch 3/5, Loss: 14.8254, F1 Score: 0.9224, Accuracy: 0.8556
Epoch 4/5, Loss: 14.8023, F1 Score: 0.9228, Accuracy: 0.8558
Epoch 5/5, Loss: 14.7793, F1 Score: 0.9230, Accuracy: 0.8561


In [10]:
model = DeeperCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 0.3328, F1 Score: 0.9325, Accuracy: 0.8788
Epoch 2/5, Loss: 0.1802, F1 Score: 0.9638, Accuracy: 0.9356
Epoch 3/5, Loss: 0.1055, F1 Score: 0.9795, Accuracy: 0.9626
Epoch 4/5, Loss: 0.0719, F1 Score: 0.9875, Accuracy: 0.9764
Epoch 5/5, Loss: 0.0508, F1 Score: 0.9927, Accuracy: 0.9849


In [21]:
torch.save(model.state_dict(), "/home/unai/Documents/Uni/Año4/Deusto/SI/ExportedModels/v1.pth")

In [17]:
model = NotsoDeepCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
simple_train_model(model=model,criterion=criterion,optimizer=optimizer,epochs=5)

Epoch 1/5, Loss: 0.4880, F1 Score: 0.9217, Accuracy: 0.8627
Epoch 2/5, Loss: 0.2173, F1 Score: 0.9553, Accuracy: 0.9202
Epoch 3/5, Loss: 0.1209, F1 Score: 0.9788, Accuracy: 0.9611
Epoch 4/5, Loss: 0.0655, F1 Score: 0.9902, Accuracy: 0.9809
Epoch 5/5, Loss: 0.0490, F1 Score: 0.9936, Accuracy: 0.9864


In [18]:
torch.save(model.state_dict(), "/home/unai/Documents/Uni/Año4/Deusto/SI/ExportedModels/v3.pth")

## Testing the trained model

In [11]:
model = NotsoDeepCNN()
model.load_state_dict(torch.load("v3.pth", weights_only=True))
model.eval()

NotsoDeepCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=40, bias=True)
  (fc3): Linear(in_features=40, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [16]:
def get_prediction(image):
    outputs = model.forward(image)
    outputs = outputs.detach().cpu().numpy()
    predicted_class = np.round(outputs)
    return predicted_class[0][0]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
y_true = []
y_pred = []
for i, data in tqdm(enumerate(test_loader, 0)):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)
    y_true.append(int(labels[0]))
    y_pred.append(get_prediction(inputs))



7580it [05:25, 23.32it/s]

Confusion matrix:
 [[ 241   73]
 [1347 5919]]


In [22]:
print("Confusion matrix:\n",
      confusion_matrix(y_pred=y_pred,y_true=y_true),
      "\n\nF1 Score:",f1_score(y_pred=y_pred,y_true=y_true))

Confusion matrix:
 [[ 241   73]
 [1347 5919]] 

F1 Score: 0.8928948559360386
